In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

# .h5 Datei öffnen
with h5py.File('transients_S01_all.h5', 'r') as f:
    def print_structure(name, obj):
        print(name)

    f.visititems(print_structure)

    dataset = f['/transients/S01/1/Normal/i_data/4999']
    data = dataset[:]

plt.plot(data)
plt.title('1D-Daten aus .h5 Datei')
plt.xlabel('Index')
plt.ylabel('Wert')
plt.grid(True)
plt.show()


In [ ]:
import h5py
import numpy as np
import random

input_file = 'transients_S01_all.h5'
output_file = 'transients_cleaned.h5'

relay_data = []
normal_data = []

with h5py.File(input_file, 'r') as f:
    for x in range(1, 11):  # Pakete 1 bis 10
        relay_path = f'transients/S01/{x}/Relay/q_data'
        normal_path = f'transients/S01/{x}/Normal/q_data'

        for y in f[relay_path]:
            data = f[relay_path][y][()]
            if len(data) <= 151:
                relay_data.append(data)

        for y in f[normal_path]:
            data = f[normal_path][y][()]
            if len(data) <= 151:
                normal_data.append(data)

random.shuffle(relay_data)
random.shuffle(normal_data)

relay_data = relay_data[:50000]
normal_data = normal_data[:50000]

with h5py.File(output_file, 'w') as f_out:
    for i, data in enumerate(relay_data):
        dset_path = f'transients/S01/1/Relay/q_data/{i}'
        f_out.create_dataset(dset_path, data=data)

    for i, data in enumerate(normal_data):
        dset_path = f'transients/S01/1/Normal/q_data/{i}'
        f_out.create_dataset(dset_path, data=data)


In [ ]:
import h5py
import numpy as np

input_file = 'transients_cleaned.h5'
output_file = 'transients_cleaned_padded.h5'

target_length = 151

def pad_data(data, target_length):
    """Füllt das Array mit Nullen bis zur Ziel-Länge"""
    if len(data) < target_length:
        padded = np.zeros(target_length, dtype=data.dtype)
        padded[:len(data)] = data
        return padded
    return data

with h5py.File(input_file, 'r') as f_in, h5py.File(output_file, 'w') as f_out:
    for category in ['Relay', 'Normal']:
        for i in range(50000):
            path = f'transients/S01/1/{category}/q_data/{i}'
            try:
                data = f_in[path][()]
                padded_data = pad_data(data, target_length)
                f_out.create_dataset(path, data=padded_data)
            except KeyError:
                print(f"Pfad nicht gefunden: {path}, überspringe.")
            except Exception as e:
                print(f"Fehler bei {path}: {e}")


In [ ]:
import h5py

file_path = 'transients_cleaned_padded.h5'

with h5py.File(file_path, 'r') as f:
    relay_group = f.get('transients/S01/1/Relay/q_data')
    normal_group = f.get('transients/S01/1/Normal/q_data')

    if relay_group is not None:
        relay_count = len(relay_group)
        print(f"Anzahl Relay-Daten: {relay_count}")
    else:
        print("Relay-Gruppe nicht gefunden.")

    if normal_group is not None:
        normal_count = len(normal_group)
        print(f"Anzahl Normal-Daten: {normal_count}")
    else:
        print("Normal-Gruppe nicht gefunden.")
